In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import classification_report, accuracy_score

def classify_volatilities(forecasted_volatility_results, actual_volatility_results, threshold=None):
    
    # Prepare the data
    companies = list(forecasted_volatility_results.keys())
    forecasted_volatilities = list(forecasted_volatility_results.values())
    actual_volatilities_values = list(actual_volatility_results.values())

    # Calculate the mean of forecasted volatilities if threshold is not provided
    if threshold is None:
        threshold = np.mean(forecasted_volatilities)
    print(f'Using threshold: {threshold}')

    # Define the target variable based on the threshold
    targets = [1 if vol > threshold else 0 for vol in forecasted_volatilities]

    # Create a DataFrame
    data = {
        'Company': companies,
        'Forecasted_Volatility': forecasted_volatilities,
        'Actual_Volatility': actual_volatilities_values,
        'Target': targets
    }
    df = pd.DataFrame(data)

    # Features and target
    X = df[['Forecasted_Volatility']]
    y = df['Target']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train SVM model
    model = svm.SVC(kernel='linear', C=1.0, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Evaluate model
    y_pred = model.predict(X_test_scaled)
    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(classification_report(y_test, y_pred))

    # Classify all companies
    X_scaled = scaler.transform(X)
    df['Class'] = model.predict(X_scaled)

    # Display the classified companies
    print(df)

    # Create lists of high and low volatility stocks
    high_volatility_stocks = df[df['Class'] == 1]['Company'].tolist()
    low_volatility_stocks = df[df['Class'] == 0]['Company'].tolist()

    print("High Volatility Stocks:", high_volatility_stocks)
    print("Low Volatility Stocks:", low_volatility_stocks)

    return df, high_volatility_stocks, low_volatility_stocks

